# Competitiveness in 5 Leagues #

In [1]:
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

**Method 2 for scraping tables** *(better for when scraping tables that already exist in website)*

In [2]:
a = ['https://fbref.com/en/comps/22/1558/2017-Major-League-Soccer-Stats', 'https://fbref.com/en/comps/22/1759/2018-Major-League-Soccer-Stats', 'https://fbref.com/en/comps/22/2798/2019-Major-League-Soccer-Stats']
print(a)

['https://fbref.com/en/comps/22/1558/2017-Major-League-Soccer-Stats', 'https://fbref.com/en/comps/22/1759/2018-Major-League-Soccer-Stats', 'https://fbref.com/en/comps/22/2798/2019-Major-League-Soccer-Stats']


## MLS Standings

In [3]:
mls = {}
for num,link in enumerate(a):    
    mls[num] = pd.read_html(link,header=0)
print(mls[0][0])

    Rk            Squad  MP   W   D   L  GF  GA  GD  Pts  Attendance  \
0    1       Toronto FC  34  20   9   5  74  37  37   69       27647   
1    2            NYCFC  34  16   9   9  56  43  13   57       22321   
2    3     Chicago Fire  34  16   7  11  61  47  14   55       17383   
3    4      Atlanta Utd  34  15  10   9  70  40  30   55       48200   
4    5    Columbus Crew  34  16   6  12  53  49   4   54       15439   
5    6     NY Red Bulls  34  14   8  12  53  47   6   50       21175   
6    7      New England  34  13   6  15  53  61  -8   45       19367   
7    8     Philadelphia  34  11   9  14  50  47   3   42       16812   
8    9  Montreal Impact  34  11   6  17  52  58  -6   39       20046   
9   10     Orlando City  34  10   9  15  39  58 -19   39       25028   
10  11      D.C. United  34   9   5  20  31  60 -29   32       17904   

                        Top Team Scorer    Goalkeeper                    Notes  
0               Sebastian Giovinco - 16     Alex Bono 

In [4]:
# To check how many tables are in each url
print(len(mls[0]))
print(len(mls[1]))
print(len(mls[2]))

14
26
26


In [5]:
# To view first two rows of each table

#for table in mls[0]:
 #   print(table[:1])

In [5]:
mls_17_east = mls[0][0].iloc[:,:-3].copy()    #Not including rows that are irrelevant
mls_17_west = mls[0][2].iloc[:,:-3].copy()
mls_18_east = mls[1][0].iloc[:,:-3].copy()
mls_18_west = mls[1][2].iloc[:,:-3].copy()
mls_19_east = mls[2][0].iloc[:,:-3].copy()
mls_19_west = mls[2][2].iloc[:,:-3].copy()

In [9]:
mls_17_east_copy = mls_17_east.copy()
mls_17_west_copy = mls_17_west.copy()
mls_18_east_copy = mls_18_east.copy()
mls_18_west_copy = mls_18_west.copy()
mls_19_east_copy = mls_19_east.copy()
mls_19_west_copy = mls_19_west.copy()

In [10]:
mls_19_east_copy['Squad']

0               NYCFC
1         Atlanta Utd
2        Philadelphia
3          Toronto FC
4         D.C. United
5        NY Red Bulls
6         New England
7        Chicago Fire
8     Montreal Impact
9       Columbus Crew
10       Orlando City
11      FC Cincinnati
Name: Squad, dtype: object

#### Do not use .loc()/iloc() when performing arithmetic to insert into dataframe

In [11]:
def add_cols_to_standings(df,col,new_col1,new_col2,new_col3,new_col4):    # will work because only need 'Pts' column
    new_col1 = col.diff().fillna(0)
    df.insert(10,"ptd",new_col1,True)

    new_col2 = [col[:2].diff().fillna(0).sum()+(df['ptd'][2])*.5] *len(df['Squad'])
    df.insert(11,'ptd_top_2_and_a_half',new_col2,True)

    new_col3 = [col[-3:].diff().fillna(0).sum()]*len(df['Squad'])
    df.insert(12,'ptd_bottom_3',new_col3,True)

    new_col4 = [col[-2:].sum()]*len(df['Squad'])
    df.insert(10,"pts_bottom_2",new_col4,True)

In [12]:
add_cols_to_standings(mls_17_east_copy,mls_17_east_copy['Pts'],'ptd','ptd_top_2_and_a_half','ptd_bottom_3','pts_bottom_2')
add_cols_to_standings(mls_17_west_copy,mls_17_west_copy['Pts'],'ptd','ptd_top_2_and_a_half','ptd_bottom_3','pts_bottom_2')
add_cols_to_standings(mls_18_east_copy,mls_18_east_copy['Pts'],'ptd','ptd_top_2_and_a_half','ptd_bottom_3','pts_bottom_2')
add_cols_to_standings(mls_18_west_copy,mls_18_west_copy['Pts'],'ptd','ptd_top_2_and_a_half','ptd_bottom_3','pts_bottom_2')
add_cols_to_standings(mls_19_east_copy,mls_19_east_copy['Pts'],'ptd','ptd_top_2_and_a_half','ptd_bottom_3','pts_bottom_2')
add_cols_to_standings(mls_19_west_copy,mls_19_west_copy['Pts'],'ptd','ptd_top_2_and_a_half','ptd_bottom_3','pts_bottom_2')

In [13]:
mls_19_east_copy.drop(['xG','xGA','xGD','xGD/90'],axis=1,inplace=True)
mls_19_west_copy.drop(['xG','xGA','xGD','xGD/90'],axis=1,inplace=True)
mls_18_east_copy.drop(['xG','xGA','xGD','xGD/90'],axis=1,inplace=True)
mls_18_west_copy.drop(['xG','xGA','xGD','xGD/90'],axis=1,inplace=True)

In [92]:
mls_19_east_copy

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,pts_bottom_2,ptd,ptd_top_2_and_a_half,ptd_bottom_3,Attendance,pk_top_2,ptd_2_and_a_half_score,ptd_bottom_3_score,pts_bottom_2_score,pk_top_2_score
0,1,NYCFC,34,18,10,6,63,42,21,64,61,0.0,-7.5,-14.0,21107,14.0,4.0,2.0,5.0,4.0
1,2,Atlanta Utd,34,18,4,12,58,43,15,58,61,-6.0,-7.5,-14.0,52510,14.0,0.0,0.0,0.0,0.0
2,3,Philadelphia,34,16,7,11,58,50,8,55,61,-3.0,-7.5,-14.0,17111,14.0,0.0,0.0,0.0,0.0
3,4,Toronto FC,34,13,11,10,57,52,5,50,61,-5.0,-7.5,-14.0,25048,14.0,0.0,0.0,0.0,0.0
4,5,D.C. United,34,13,11,10,42,38,4,50,61,0.0,-7.5,-14.0,17744,14.0,0.0,0.0,0.0,0.0
5,6,NY Red Bulls,34,14,6,14,53,51,2,48,61,-2.0,-7.5,-14.0,17751,14.0,0.0,0.0,0.0,0.0
6,7,New England,34,11,12,11,50,57,-7,45,61,-3.0,-7.5,-14.0,16737,14.0,0.0,0.0,0.0,0.0
7,8,Chicago Fire,34,10,12,12,55,47,8,42,61,-3.0,-7.5,-14.0,12324,14.0,0.0,0.0,0.0,0.0
8,9,Montreal Impact,34,12,5,17,47,60,-13,41,61,-1.0,-7.5,-14.0,16171,14.0,0.0,0.0,0.0,0.0
9,10,Columbus Crew,34,10,8,16,39,47,-8,38,61,-3.0,-7.5,-14.0,14856,14.0,0.0,0.0,0.0,0.0


In [15]:
mls_east_teams = mls_19_east['Squad']
mls_west_teams = mls_19_west['Squad']
mls_east_teams

0               NYCFC
1         Atlanta Utd
2        Philadelphia
3          Toronto FC
4         D.C. United
5        NY Red Bulls
6         New England
7        Chicago Fire
8     Montreal Impact
9       Columbus Crew
10       Orlando City
11      FC Cincinnati
Name: Squad, dtype: object

## MLS Fixtures

In [6]:
mls_fixtures = ['https://fbref.com/en/comps/22/1558/schedule/2017-Major-League-Soccer-Scores-and-Fixtures','https://fbref.com/en/comps/22/1759/schedule/2018-Major-League-Soccer-Scores-and-Fixtures','https://fbref.com/en/comps/22/2798/schedule/2019-Major-League-Soccer-Scores-and-Fixtures']

In [7]:
mls_results = {}
for num,link in enumerate(mls_fixtures):    
    mls_results[num] = pd.read_html(link,header=0)
print(mls_results[0][1])

     Day        Date   Time              Home Score           Away Attendance  \
0    Fri  2017-03-03  18:30  Portland Timbers   5–1  Minnesota Utd      21144   
1    Sat  2017-03-04  14:00     Columbus Crew   1–1   Chicago Fire      15023   
2    Sat  2017-03-04  13:00         LA Galaxy   1–2      FC Dallas      23554   
3    Sat  2017-03-04  14:30    Real Salt Lake   0–0     Toronto FC      19519   
4    Sat  2017-03-04  16:00   Colorado Rapids   1–0    New England      16126   
..   ...         ...    ...               ...   ...            ...        ...   
383  Sun  2017-10-22  15:00    Houston Dynamo   3–0   Chicago Fire      20456   
384  Sun  2017-10-22  16:00      Philadelphia   6–1   Orlando City      18595   
385  Sun  2017-10-22  16:00   Montreal Impact   2–3    New England      20681   
386  Sun  2017-10-22  16:00             NYCFC   2–2  Columbus Crew      20113   
387  Sun  2017-10-22  16:00       D.C. United   1–2   NY Red Bulls      41418   

                           

In [8]:
mls_17_fixtures = mls_results[0][1].iloc[:,:-3].copy()
mls_18_fixtures = mls_results[1][1].iloc[:,:-3].copy()
mls_19_fixtures = mls_results[2][1].iloc[:,:-3].copy()

In [17]:
mls_19_fixtures.drop(['xG','xG.1'],axis=1,inplace=True)    # NEED TO DROP 2 COLUMNS SIMULTANEOUSLY
mls_18_fixtures.drop(['xG','xG.1'],axis=1,inplace=True)

In [18]:
mls_19_fixtures

,Day,Date,Time,Home,Score,Away,Attendance,Venue
0,Sat,2019-03-02,13:00,Philadelphia,1–3,Toronto FC,16498,Subaru Park
1,Sat,2019-03-02,14:30,Orlando City,2–2,NYCFC,25527,Exploria Stadium
2,Sat,2019-03-02,15:30,FC Dallas,1–1,New England,14327,Toyota Stadium
3,Sat,2019-03-02,16:30,Columbus Crew,1–1,NY Red Bulls,17931,MAPFRE Stadium
4,Sat,2019-03-02,15:00,Vancouver,2–3,Minnesota Utd,27837,BC Place
...,...,...,...,...,...,...,...,...
419,Sun,2019-10-06,16:00,Atlanta Utd,3–1,New England,44617,Mercedes-Benz Stadium
420,Sun,2019-10-06,16:00,Toronto FC,1–0,Columbus Crew,23692,BMO Field
421,Sun,2019-10-06,16:00,Orlando City,2–5,Chicago Fire,22433,Exploria Stadium
422,Sun,2019-10-06,16:00,D.C. United,0–0,FC Cincinnati,18732,Audi Field


In [19]:
# Create 2 DataFrames one for each Conference

mls_17_east_matches = mls_17_fixtures.copy()
mls_17_west_matches = mls_17_fixtures.copy()
mls_18_east_matches = mls_18_fixtures.copy()
mls_18_west_matches = mls_18_fixtures.copy()
mls_19_east_matches = mls_19_fixtures.copy()
mls_19_west_matches = mls_19_fixtures.copy()

In [20]:
mls_19_east.iloc[:2,1].copy()

0          NYCFC
1    Atlanta Utd
Name: Squad, dtype: object

In [21]:
e_17_champ = []
e_17_top_2 = []
e_17_top_3 = []
e_17_top_4 = []
e_17_bottom_3 = []

w_17_champ = []
w_17_top_2 = []
w_17_top_3 = []
w_17_top_4 = []
w_17_bottom_3 = []

e_18_champ = []
e_18_top_2 = []
e_18_top_3 = []
e_18_top_4 = []
e_18_bottom_3 = []

w_18_champ = []
w_18_top_2 = []
w_18_top_3 = []
w_18_top_4 = []
w_18_bottom_3 = []

e_19_champ =[]
e_19_top_2 =[]
e_19_top_3 =[]
e_19_top_4 =[]
e_19_bottom_3 =[]

w_19_champ =[]
w_19_top_2 =[]
w_19_top_3 =[]
w_19_top_4 =[]
w_19_bottom_3 = []

In [22]:
# CREATING SERIES OF SUBSETS OF ALL TEAMS
def top_teams(east,west,e_champ,e_top_2,e_top_3,e_top_4,e_bottom_3,w_champ,w_top_2,w_top_3,w_top_4,w_bottom_3):
    e_champ.extend(east.iloc[:1,1].copy())
    e_top_2.extend(east.iloc[:2,1].copy())
    e_top_3.extend(east.iloc[:3,1].copy())
    e_top_4.extend(east.iloc[:4,1].copy())
    e_bottom_3.extend(east.iloc[-3:,1].copy())

    w_champ.extend(west.iloc[:1,1].copy())
    w_top_2.extend(west.iloc[:2,1].copy())
    w_top_3.extend(west.iloc[:3,1].copy())
    w_top_4.extend(west.iloc[:4,1].copy())
    w_bottom_3.extend(west.iloc[-3:,1].copy())

In [23]:
top_teams(mls_17_east,mls_17_west,e_17_champ,e_17_top_2,e_17_top_3,e_17_top_4,e_17_bottom_3,w_17_champ,w_17_top_2,w_17_top_3,w_17_top_4,w_17_bottom_3)
top_teams(mls_18_east,mls_18_west,e_18_champ,e_18_top_2,e_18_top_3,e_18_top_4,e_18_bottom_3,w_18_champ,w_18_top_2,w_18_top_3,w_18_top_4,w_18_bottom_3)
top_teams(mls_19_east,mls_19_west,e_19_champ,e_19_top_2,e_19_top_3,e_19_top_4,e_19_bottom_3,w_19_champ,w_19_top_2,w_19_top_3,w_19_top_4,w_19_bottom_3)

In [24]:
pd.Series(e_19_top_4)

0           NYCFC
1     Atlanta Utd
2    Philadelphia
3      Toronto FC
dtype: object

In [25]:
# FUNCTION FOR SEPERATING FIXTURES BY CONFERENCES

def sep_conf(e_df,w_df):
    for index,row in e_df.iterrows():
        if (row['Home']==mls_west_teams).any() | (row['Away']==mls_west_teams).any():
            e_df.drop(index,inplace=True)
    for index,row in w_df.iterrows():
        if (row['Home']==mls_east_teams).any() | (row['Away']==mls_east_teams).any():
            w_df.drop(index,inplace=True)

In [26]:
sep_conf(mls_17_east_matches,mls_17_west_matches)
sep_conf(mls_18_east_matches,mls_18_west_matches)
sep_conf(mls_19_east_matches,mls_19_west_matches)

In [28]:
mls_19_east_matches

,Day,Date,Time,Home,Score,Away,Attendance,Venue
0,Sat,2019-03-02,13:00,Philadelphia,1–3,Toronto FC,16498,Subaru Park
1,Sat,2019-03-02,14:30,Orlando City,2–2,NYCFC,25527,Exploria Stadium
3,Sat,2019-03-02,16:30,Columbus Crew,1–1,NY Red Bulls,17931,MAPFRE Stadium
10,Sun,2019-03-03,18:00,D.C. United,2–0,Atlanta Utd,16942,Audi Field
12,Sat,2019-03-09,12:00,Chicago Fire,1–1,Orlando City,11041,SeatGeek Stadium
...,...,...,...,...,...,...,...,...
419,Sun,2019-10-06,16:00,Atlanta Utd,3–1,New England,44617,Mercedes-Benz Stadium
420,Sun,2019-10-06,16:00,Toronto FC,1–0,Columbus Crew,23692,BMO Field
421,Sun,2019-10-06,16:00,Orlando City,2–5,Chicago Fire,22433,Exploria Stadium
422,Sun,2019-10-06,16:00,D.C. United,0–0,FC Cincinnati,18732,Audi Field


In [29]:
mls_17_east_matches_copy = mls_17_east_matches.copy()
mls_17_west_matches_copy = mls_17_west_matches.copy()
mls_18_east_matches_copy = mls_18_east_matches.copy()
mls_18_west_matches_copy = mls_18_west_matches.copy()
mls_19_east_matches_copy = mls_19_east_matches.copy()
mls_19_west_matches_copy = mls_19_west_matches.copy()

In [30]:
# FUNCTION FOR CLEAINING FIXTURES TABLES

def clean_fixtures(east,west,e_champ,e_top_2,e_top_3,e_top_4,e_bottom_3,w_champ,w_top_2,w_top_3,w_top_4,w_bottom_3):

    # EASTERN CONFERENCE

    #Losses by 2+ goals by not top 2-3, USE .all() WHEN DOING NOT EQUAL TO
    for index,row in east.iterrows():
        if ((row['Home']==pd.Series(e_top_2)).any() & (row['Away']!=pd.Series(e_top_2)).all()) and int(row['Score'][2])-int(row['Score'][0]) >=2:
            east.loc[index,'top_2_L_by2goals']=0.5  
        elif ((row['Away']==pd.Series(e_top_2)).any() & (row['Home']!=pd.Series(e_top_2)).all()) and int(row['Score'][2])-int(row['Score'][0]) <=-2:
            east.loc[index,'top_2_L_by2goals']=0.5

    for index,row in east.iterrows():
        if ((row['Home']==pd.Series(e_top_3)).any() & (row['Away']!=pd.Series(e_top_3)).all()) and int(row['Score'][2])-int(row['Score'][0]) >=2:
            east.loc[index,'top_3_L_by2goals']=0.5
        elif ((row['Away']==pd.Series(e_top_3)).any() & (row['Home']!=pd.Series(e_top_3)).all()) and int(row['Score'][2])-int(row['Score'][0]) <=-2:
            east.loc[index,'top_3_L_by2goals']=0.5

    #Losses against bottom 3
    for index,row in east.iterrows():
        if ((row['Home']==pd.Series(e_top_2)).any() & (row['Away']==pd.Series(e_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) >=1:
            east.loc[index,'top_2_L_bybottom3']=0.5
        elif ((row['Away']==pd.Series(e_top_2)).any() & (row['Home']==pd.Series(e_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) <=-1:
            east.loc[index,'top_2_L_bybottom3']=0.5

    for index,row in east.iterrows():
        if ((row['Home']==pd.Series(e_top_3)).any() & (row['Away']==pd.Series(e_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) >=1:
            east.loc[index,'top_3_L_bybottom3']=0.5
        elif ((row['Away']==pd.Series(e_top_3)).any() & (row['Home']==pd.Series(e_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) <=-1:
            east.loc[index,'top_3_L_bybottom3']=0.5

    #Champ Losses against top 4, THEY DIDN'T LOSE ANY FOR THIS CONDITION
    for index,row in east.iterrows():
        if ((row['Home']==pd.Series(e_champ)).any() & (row['Away']==pd.Series(e_top_4)).any()) and int(row['Score'][2])-int(row['Score'][0]) >=1:
            east.loc[index,'champ_L_bytop4']=1
        elif ((row['Away']==pd.Series(e_champ)).any() & (row['Home']==pd.Series(e_top_4)).any()) and int(row['Score'][2])-int(row['Score'][0]) <=-1:
            east.loc[index,'champ_L_bytop4']=1


    # WESTERN CONFERENCE

    #Losses by 2+ goals by not top 2-3, USE .all() WHEN DOING NOT EQUAL TO
    for index,row in west.iterrows():
        if ((row['Home']==pd.Series(w_top_2)).any() & (row['Away']!=pd.Series(w_top_2)).all()) and int(row['Score'][2])-int(row['Score'][0]) >=2:
            west.loc[index,'top_2_L_by2goals']=0.5  
        elif ((row['Away']==pd.Series(w_top_2)).any() & (row['Home']!=pd.Series(w_top_2)).all()) and int(row['Score'][2])-int(row['Score'][0]) <=-2:
            west.loc[index,'top_2_L_by2goals']=0.5

    for index,row in west.iterrows():
        if ((row['Home']==pd.Series(w_top_3)).any() & (row['Away']!=pd.Series(w_top_3)).all()) and int(row['Score'][2])-int(row['Score'][0]) >=2:
            west.loc[index,'top_3_L_by2goals']=0.5
        elif ((row['Away']==pd.Series(w_top_3)).any() & (row['Home']!=pd.Series(w_top_3)).all()) and int(row['Score'][2])-int(row['Score'][0]) <=-2:
            west.loc[index,'top_3_L_by2goals']=0.5

    #Losses against bottom 3
    for index,row in west.iterrows():
        if ((row['Home']==pd.Series(w_top_2)).any() & (row['Away']==pd.Series(w_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) >=1:
            west.loc[index,'top_2_L_bybottom3']=0.5
        elif ((row['Away']==pd.Series(w_top_2)).any() & (row['Home']==pd.Series(w_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) <=-1:
            west.loc[index,'top_2_L_bybottom3']=0.5

    for index,row in west.iterrows():
        if ((row['Home']==pd.Series(w_top_3)).any() & (row['Away']==pd.Series(w_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) >=1:
            west.loc[index,'top_3_L_bybottom3']=0.5
        elif ((row['Away']==pd.Series(w_top_3)).any() & (row['Home']==pd.Series(w_bottom_3)).any()) and int(row['Score'][2])-int(row['Score'][0]) <=-1:
            west.loc[index,'top_3_L_bybottom3']=0.5

    #Champ Losses against top 4, THEY DIDN'T LOSE ANY FOR THIS CONDITION
    for index,row in west.iterrows():
        if ((row['Home']==pd.Series(w_champ)).any() & (row['Away']==pd.Series(w_top_4)).any()) and int(row['Score'][2])-int(row['Score'][0]) >=1:
            west.loc[index,'champ_L_bytop4']=1
        elif ((row['Away']==pd.Series(w_champ)).any() & (row['Home']==pd.Series(w_top_4)).any()) and int(row['Score'][2])-int(row['Score'][0]) <=-1:
            west.loc[index,'champ_L_bytop4']=1

In [31]:
clean_fixtures(mls_17_east_matches_copy,mls_17_west_matches_copy,e_17_champ,e_17_top_2,e_17_top_3,e_17_top_4,e_17_bottom_3,w_17_champ,w_17_top_2,w_17_top_3,w_17_top_4,w_17_bottom_3)
clean_fixtures(mls_18_east_matches_copy,mls_18_west_matches_copy,e_18_champ,e_18_top_2,e_18_top_3,e_18_top_4,e_18_bottom_3,w_18_champ,w_18_top_2,w_18_top_3,w_18_top_4,w_18_bottom_3)
clean_fixtures(mls_19_east_matches_copy,mls_19_west_matches_copy,e_19_champ,e_19_top_2,e_19_top_3,e_19_top_4,e_19_bottom_3,w_19_champ,w_19_top_2,w_19_top_3,w_19_top_4,w_19_bottom_3)

In [32]:
mls_18_east_matches_copy.loc[mls_18_east_matches_copy['top_2_L_bybottom3']== 0.5]

,Day,Date,Time,Home,Score,Away,Attendance,Venue,top_2_L_by2goals,top_3_L_by2goals,top_2_L_bybottom3,top_3_L_bybottom3,champ_L_bytop4
36,Sat,2018-03-31,13:00,Orlando City,4–3,NY Red Bulls,23257,Exploria Stadium,NaN,NaN,0.5,0.5,NaN
70,Sat,2018-04-21,15:30,NY Red Bulls,1–2,Chicago Fire,18784,Red Bull Arena,NaN,NaN,0.5,0.5,NaN
404,Sun,2018-10-28,16:30,Toronto FC,4–1,Atlanta Utd,23895,BMO Field,0.5,0.5,0.5,0.5,NaN


In [38]:
mls_17_east_matches_copy = mls_17_east_matches_copy.fillna(0)
mls_17_west_matches_copy = mls_17_west_matches_copy.fillna(0)
mls_18_east_matches_copy = mls_18_east_matches_copy.fillna(0)
mls_18_west_matches_copy = mls_18_west_matches_copy.fillna(0)
mls_19_east_matches_copy = mls_19_east_matches_copy.fillna(0)
mls_19_west_matches_copy = mls_19_west_matches_copy.fillna(0)

In [39]:
for index,row in mls_18_east_matches.iterrows():
    if ((row['Home']==pd.Series(e_18_top_3).any()) & (row['Away']!=pd.Series(e_18_bottom_3).all())) and int(row['Score'][2])-int(row['Score'][0]) >=2:
        print(row)
    elif ((row['Away']==pd.Series(e_18_top_3).any()) & (row['Home']!=pd.Series(e_18_bottom_3).all())) and int(row['Score'][2])-int(row['Score'][0]) <=-2:
        print(row)

Day                       Sat
Date               2018-09-01
Time                    19:30
Home          Montreal Impact
Score                     3–0
Away             NY Red Bulls
Attendance              18458
Venue          Saputo Stadium
Name: 313, dtype: object


In [93]:
# Scraping Performance tables to get total pk's awarded to the top 2 teams

mls_17_performance = mls[0][4].copy()
mls_18_performance = mls[1][4].copy()
mls_19_performance = mls[2][4].copy()

#### Do not use .loc()/iloc() when performing arithmetic to insert into dataframe

In [94]:
def add_pk(performance,e_standing,w_standing):
    new_header = performance.iloc[0]
    performance = performance[1:]
    performance.columns = new_header
    performance = performance.iloc[:,11:13].astype(int).sort_values(['PKatt'],ascending=False)
    pk_top_2e = [performance['PKatt'][:4].astype(int).sum()/2]*len(e_standing['Squad'])    #divided by 2 for the 2 different Conferences
    pk_top_2w = [performance['PKatt'][:4].astype(int).sum()/2]*len(w_standing['Squad'])
    e_standing.insert(15,'pk_top_2',pk_top_2e,True)
    w_standing.insert(15,'pk_top_2',pk_top_2w,True)

In [35]:
add_pk(mls_17_performance,mls_17_east_copy,mls_17_west_copy)
add_pk(mls_18_performance,mls_18_east_copy,mls_18_west_copy)
add_pk(mls_19_performance,mls_19_east_copy,mls_19_west_copy)

In [36]:
#mls_18_west_copy = mls_18_west_copy.drop(mls_18_west_copy.columns[-1:],axis=1)
mls_18_east_copy

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,pts_bottom_2,ptd,ptd_top_2_and_a_half,ptd_bottom_3,Attendance,pk_top_2
0,1,NY Red Bulls,34,22,5,7,62,33,29,71,60,0.0,-8.5,-8.0,18601,15.0
1,2,Atlanta Utd,34,21,6,7,70,44,26,69,60,-2.0,-8.5,-8.0,53002,15.0
2,3,NYCFC,34,16,8,10,59,45,14,56,60,-13.0,-8.5,-8.0,23211,15.0
3,4,D.C. United,34,14,9,11,60,50,10,51,60,-5.0,-8.5,-8.0,17635,15.0
4,5,Columbus Crew,34,14,9,11,43,45,-2,51,60,0.0,-8.5,-8.0,12447,15.0
5,6,Philadelphia,34,15,5,14,49,50,-1,50,60,-1.0,-8.5,-8.0,16518,15.0
6,7,Montreal Impact,34,14,4,16,47,53,-6,46,60,-4.0,-8.5,-8.0,18569,15.0
7,8,New England,34,10,11,13,49,55,-6,41,60,-5.0,-8.5,-8.0,18347,15.0
8,9,Toronto FC,34,10,6,18,59,64,-5,36,60,-5.0,-8.5,-8.0,26628,15.0
9,10,Chicago Fire,34,8,8,18,48,61,-13,32,60,-4.0,-8.5,-8.0,14806,15.0


In [97]:
for i in np.linspace(-16,-10,60,endpoint=False):
    print(i)

-16.0
-15.9
-15.8
-15.7
-15.6
-15.5
-15.4
-15.3
-15.2
-15.1
-15.0
-14.9
-14.8
-14.7
-14.6
-14.5
-14.4
-14.3
-14.2
-14.1
-14.0
-13.9
-13.8
-13.7
-13.6
-13.5
-13.4
-13.3
-13.2
-13.1
-13.0
-12.9
-12.8
-12.7
-12.6
-12.5
-12.4
-12.3
-12.2
-12.1
-12.0
-11.899999999999999
-11.8
-11.7
-11.6
-11.5
-11.399999999999999
-11.3
-11.2
-11.1
-11.0
-10.899999999999999
-10.8
-10.7
-10.6
-10.5
-10.399999999999999
-10.3
-10.2
-10.1


In [98]:
def add_scores(east,west):
    # EASTERN CONFERENCE
    if east.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-6,0,60,endpoint=False):
        east.loc[0,'ptd_2_and_a_half_score'] = 5
    elif east.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-8,-6,20,endpoint=False):
            east.loc[0,'ptd_2_and_a_half_score'] = 4
    elif east.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-11,-8,30,endpoint=False):
            east.loc[0,'ptd_2_and_a_half_score'] = 3
    elif east.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-13,-11,20,endpoint=False):
            east.loc[0,'ptd_2_and_a_half_score'] = 2
    elif east.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-16,-13,30,endpoint=False):
            east.loc[0,'ptd_2_and_a_half_score'] = 1
    elif east.loc[0,'ptd_top_2_and_a_half'] <-16:
            east.loc[0,'ptd_2_and_a_half_score'] = 0
            
    if east.loc[0,'ptd_bottom_3'] in np.linspace(-7,0,70,endpoint=False):
        east.loc[0,'ptd_bottom_3_score'] = 5
    elif east.loc[0,'ptd_bottom_3'] in np.linspace(-10,-7,30,endpoint=False):
            east.loc[0,'ptd_bottom_3_score'] = 4
    elif east.loc[0,'ptd_bottom_3'] in np.linspace(-12,-10,20,endpoint=False):
            east.loc[0,'ptd_bottom_3_score'] = 3
    elif east.loc[0,'ptd_bottom_3'] in np.linspace(-14,-12,20,endpoint=False):
            east.loc[0,'ptd_bottom_3_score'] = 2
    elif east.loc[0,'ptd_bottom_3'] in np.linspace(-17,-14,30,endpoint=False):
            east.loc[0,'ptd_bottom_3_score'] = 1
    elif east.loc[0,'ptd_bottom_3'] <-17:
            east.loc[0,'ptd_bottom_3_score'] = 0
            
    if east.loc[0,'pts_bottom_2'] in np.linspace(50,55,50,endpoint=False):
        east.loc[0,'pts_bottom_2_score'] = 3
    elif east.loc[0,'pts_bottom_2'] in np.linspace(55,58,30,endpoint=False):
            east.loc[0,'pts_bottom_2_score'] = 4
    elif east.loc[0,'pts_bottom_2'] >=58:
            east.loc[0,'pts_bottom_2_score'] = 5
            
    if east.loc[0,'pk_top_2'] in np.linspace(4,13,90,endpoint=False):
        east.loc[0,'pk_top_2_score'] = 5
    elif east.loc[0,'pk_top_2'] in np.linspace(13,15,20,endpoint=False):
            east.loc[0,'pk_top_2_score'] = 4
    elif east.loc[0,'pk_top_2'] in np.linspace(15,18,30,endpoint=False):
            east.loc[0,'pk_top_2_score'] = 3
    elif east.loc[0,'pk_top_2'] in np.linspace(18,20,20,endpoint=False):
            east.loc[0,'pk_top_2_score'] = 2
    elif east.loc[0,'pk_top_2'] in np.linspace(21,24,30,endpoint=False):
            east.loc[0,'pk_top_2_score'] = 1
    elif east.loc[0,'pk_top_2'] >=24:
            east.loc[0,'pk_top_2_score'] = 0   
            
            
    # WESTERN CONFERENCE
    if west.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-6,0,60,endpoint=False):
        west.loc[0,'ptd_2_and_a_half_score'] = 5
    elif west.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-8,-6,20,endpoint=False):
            west.loc[0,'ptd_2_and_a_half_score'] = 4
    elif west.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-11,-8,30,endpoint=False):
            west.loc[0,'ptd_2_and_a_half_score'] = 3
    elif west.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-13,-11,20,endpoint=False):
            west.loc[0,'ptd_2_and_a_half_score'] = 2
    elif west.loc[0,'ptd_top_2_and_a_half'] in np.linspace(-16,-13,30,endpoint=False):
            west.loc[0,'ptd_2_and_a_half_score'] = 1
    elif west.loc[0,'ptd_top_2_and_a_half'] <-16:
            west.loc[0,'ptd_2_and_a_half_score'] = 0
            
    if west.loc[0,'ptd_bottom_3'] in np.linspace(-7,0,70,endpoint=False):
        west.loc[0,'ptd_bottom_3_score'] = 5
    elif west.loc[0,'ptd_bottom_3'] in np.linspace(-10,-7,30,endpoint=False):
            west.loc[0,'ptd_bottom_3_score'] = 4
    elif west.loc[0,'ptd_bottom_3'] in np.linspace(-12,-10,20,endpoint=False):
            west.loc[0,'ptd_bottom_3_score'] = 3
    elif west.loc[0,'ptd_bottom_3'] in np.linspace(-14,-12,20,endpoint=False):
            west.loc[0,'ptd_bottom_3_score'] = 2
    elif west.loc[0,'ptd_bottom_3'] in np.linspace(-17,-14,30,endpoint=False):
            west.loc[0,'ptd_bottom_3_score'] = 1
    elif west.loc[0,'ptd_bottom_3'] <-17:
            west.loc[0,'ptd_bottom_3_score'] = 0
            
    if west.loc[0,'pts_bottom_2'] in np.linspace(50,55,50,endpoint=False):
        west.loc[0,'pts_bottom_2_score'] = 3
    elif west.loc[0,'pts_bottom_2'] in np.linspace(55,58,30,endpoint=False):
            west.loc[0,'pts_bottom_2_score'] = 4
    elif west.loc[0,'pts_bottom_2'] >=58:
            west.loc[0,'pts_bottom_2_score'] = 5
            
    if west.loc[0,'pk_top_2'] in np.linspace(4,13,90,endpoint=False):
        west.loc[0,'pk_top_2_score'] = 5
    elif west.loc[0,'pk_top_2'] in np.linspace(13,15,20,endpoint=False):
            west.loc[0,'pk_top_2_score'] = 4
    elif west.loc[0,'pk_top_2'] in np.linspace(15,18,30,endpoint=False):
            west.loc[0,'pk_top_2_score'] = 3
    elif west.loc[0,'pk_top_2'] in np.linspace(18,20,20,endpoint=False):
            west.loc[0,'pk_top_2_score'] = 2
    elif west.loc[0,'pk_top_2'] in np.linspace(21,24,30,endpoint=False):
            west.loc[0,'pk_top_2_score'] = 1
    elif west.loc[0,'pk_top_2'] >=24:
            west.loc[0,'pk_top_2_score'] = 0               

In [39]:
pd.options.mode.chained_assignment = None #to remove unnecesary warning

add_scores(mls_17_east_copy,mls_17_west_copy)
add_scores(mls_18_east_copy,mls_18_west_copy)
add_scores(mls_19_east_copy,mls_19_west_copy)

In [40]:
mls_17_east_copy = mls_17_east_copy.fillna(0)
mls_17_west_copy = mls_17_west_copy.fillna(0)
mls_18_east_copy = mls_18_east_copy.fillna(0)
mls_18_west_copy = mls_18_west_copy.fillna(0)
mls_19_east_copy = mls_19_east_copy.fillna(0)
mls_19_west_copy = mls_19_west_copy.fillna(0)

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,pts_bottom_2,ptd,ptd_top_2_and_a_half,ptd_bottom_3,Attendance,pk_top_2,ptd_2_and_a_half_score,ptd_bottom_3_score,pts_bottom_2_score,pk_top_2_score
0,1,Los Angeles FC,34,21,9,4,85,37,48,72,72,0.0,-17.5,-6.0,22251,14.0,0.0,5.0,5.0,4.0
1,2,Seattle,34,16,8,10,52,49,3,56,72,-16.0,-17.5,-6.0,40247,14.0,0.0,0.0,0.0,0.0
2,3,Real Salt Lake,34,16,5,13,46,41,5,53,72,-3.0,-17.5,-6.0,18121,14.0,0.0,0.0,0.0,0.0
3,4,Minnesota Utd,34,15,8,11,52,43,9,53,72,0.0,-17.5,-6.0,19723,14.0,0.0,0.0,0.0,0.0
4,5,LA Galaxy,34,16,3,15,58,59,-1,51,72,-2.0,-17.5,-6.0,23205,14.0,0.0,0.0,0.0,0.0
5,6,Portland Timbers,34,14,7,13,52,49,3,49,72,-2.0,-17.5,-6.0,25218,14.0,0.0,0.0,0.0,0.0
6,7,FC Dallas,34,13,9,12,54,46,8,48,72,-1.0,-17.5,-6.0,14842,14.0,0.0,0.0,0.0,0.0
7,8,San Jose,34,13,5,16,52,55,-3,44,72,-4.0,-17.5,-6.0,18781,14.0,0.0,0.0,0.0,0.0
8,9,Colorado Rapids,34,12,6,16,58,63,-5,42,72,-2.0,-17.5,-6.0,14284,14.0,0.0,0.0,0.0,0.0
9,10,Houston Dynamo,34,12,4,18,49,59,-10,40,72,-2.0,-17.5,-6.0,15674,14.0,0.0,0.0,0.0,0.0


In [41]:
#mls_19_east_copy = mls_19_east_copy.drop(mls_19_east_copy.columns[-4:],axis=1)
mls_19_west_copy

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,pts_bottom_2,ptd,ptd_top_2_and_a_half,ptd_bottom_3,Attendance,pk_top_2,ptd_2_and_a_half_score,ptd_bottom_3_score,pts_bottom_2_score,pk_top_2_score
0,1,Los Angeles FC,34,21,9,4,85,37,48,72,72,0.0,-17.5,-6.0,22251,14.0,0.0,5.0,5.0,4.0
1,2,Seattle,34,16,8,10,52,49,3,56,72,-16.0,-17.5,-6.0,40247,14.0,0.0,0.0,0.0,0.0
2,3,Real Salt Lake,34,16,5,13,46,41,5,53,72,-3.0,-17.5,-6.0,18121,14.0,0.0,0.0,0.0,0.0
3,4,Minnesota Utd,34,15,8,11,52,43,9,53,72,0.0,-17.5,-6.0,19723,14.0,0.0,0.0,0.0,0.0
4,5,LA Galaxy,34,16,3,15,58,59,-1,51,72,-2.0,-17.5,-6.0,23205,14.0,0.0,0.0,0.0,0.0
5,6,Portland Timbers,34,14,7,13,52,49,3,49,72,-2.0,-17.5,-6.0,25218,14.0,0.0,0.0,0.0,0.0
6,7,FC Dallas,34,13,9,12,54,46,8,48,72,-1.0,-17.5,-6.0,14842,14.0,0.0,0.0,0.0,0.0
7,8,San Jose,34,13,5,16,52,55,-3,44,72,-4.0,-17.5,-6.0,18781,14.0,0.0,0.0,0.0,0.0
8,9,Colorado Rapids,34,12,6,16,58,63,-5,42,72,-2.0,-17.5,-6.0,14284,14.0,0.0,0.0,0.0,0.0
9,10,Houston Dynamo,34,12,4,18,49,59,-10,40,72,-2.0,-17.5,-6.0,15674,14.0,0.0,0.0,0.0,0.0


In [89]:
mls_17_east_copy.to_csv('mls_17_east.csv',sep=',',index=False)
mls_17_west_copy.to_csv('mls_17_west.csv',sep=',',index=False)
mls_18_east_copy.to_csv('mls_18_east.csv',sep=',',index=False)
mls_18_west_copy.to_csv('mls_18_west.csv',sep=',',index=False)
mls_19_east_copy.to_csv('mls_19_east.csv',sep=',',index=False)
mls_19_west_copy.to_csv('mls_19_west.csv',sep=',',index=False)

In [90]:
mls_17_east_matches_copy.to_csv('mls_17_east_matches.csv',sep=',',index=False)
mls_17_west_matches_copy.to_csv('mls_17_west_matches.csv',sep=',',index=False)
mls_18_east_matches_copy.to_csv('mls_18_east_matches.csv',sep=',',index=False)
mls_18_west_matches_copy.to_csv('mls_18_west_matches.csv',sep=',',index=False)
mls_19_east_matches_copy.to_csv('mls_19_east_matches.csv',sep=',',index=False)
mls_19_west_matches_copy.to_csv('mls_19_west_matches.csv',sep=',',index=False)